In [ ]:
import os
from pathlib import Path
from types import SimpleNamespace

cfg = SimpleNamespace(**{})
cfg.num_folds = 5
cfg.fold = -1
cfg.gpu = "7"
os.environ['CUDA_VISIBLE_DEVICES'] = cfg.gpu

cfg.fname = 'bird_003'
cfg.seed = 2024

cfg.input_path = Path('../input')
cfg.comp_data_path = cfg.input_path / 'birdclef-2024'
cfg.save_path = Path('../checkpoints')
cfg.soundscape_path = cfg.comp_data_path / 'unlabeled_soundscapes'

cfg.logger_file = True

# image size
cfg.image_height = 224
cfg.image_width = 224

# audio
cfg.duration = 5
cfg.sr = 32000
cfg.fmin = 90
cfg.fmax = 14000
cfg.n_fft = 1536
cfg.n_mels = cfg.image_height
cfg.win_length = 1024
cfg.hop_length = int((cfg.duration * cfg.sr - cfg.win_length + cfg.n_fft) / (cfg.image_width)) + 1 

# training HP
cfg.num_epochs = 30
cfg.train_batch_size = 128
cfg.valid_batch_size = 128
cfg.workers = 2
cfg.grad_value = 2.0
cfg.grad_norm = 0.0
cfg.grad_norm_type = 2
cfg.device = "cuda"
cfg.accumulate = 1

# optimizer
cfg.lr = 7e-5
cfg.decay = 0.01
cfg.opt_beta1 = 0.9
cfg.opt_beta2 = 0.999
cfg.opt_eps = 1e-8
cfg.optimizer = 'AdamW'
cfg.no_decay = True

# scheduler
cfg.pct_start = 0.1
cfg.max_lr = 3e-3
cfg.final_div_factor = 100

# augmentations
cfg.resample_train = 10
cfg.other_samples = 1
cfg.max_shift = 1
cfg.loudness_range = 10.0

# logging
cfg.local_rank = 0
cfg.verbose=True

# model
cfg.backbone = 'efficientvit_b0.r224_in1k'
cfg.gem_pooling = False
cfg.bce = True

# tasks hp
cfg.train_model = True
cfg.num_folds = 5
cfg.pl = None
cfg.pretrained_path = None 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
import gc
import pickle as pkl

import librosa

from torch.utils.data import DataLoader, Dataset
import torchaudio
import torchaudio.transforms as T
#import torch_audiomentations as tA

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.optim import lr_scheduler, Adam, AdamW

import timm

from glob import glob
from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import roc_auc_score
from scipy.special import logit, expit

In [ ]:
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

In [ ]:
if cfg.train_model:
    checkpoint_path = cfg.save_path / cfg.fname
    if not checkpoint_path.exists():
        checkpoint_path.mkdir()
    checkpoint_path = checkpoint_path / 'exp_0'
    exp = 0
    while(checkpoint_path.exists()):
        exp += 1
        checkpoint_path = cfg.save_path / cfg.fname / ('exp_%d' % exp)
    checkpoint_path.mkdir()
    cfg.checkpoint_path = checkpoint_path
    print('saving checkpoints to', checkpoint_path)

In [ ]:
def get_logger(cfg):
    logger = getLogger(cfg.fname)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    if cfg.logger_file and cfg.train_model:
        filename = cfg.checkpoint_path / 'run.log'
        handler2 = FileHandler(filename=filename)
        handler2.setFormatter(Formatter("%(message)s"))
        logger.addHandler(handler2)
    return logger

def seed_torch(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
    if torch.backends.cudnn.is_available:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [ ]:
train = pd.read_csv(cfg.comp_data_path / 'train_metadata.csv')
train

In [ ]:
train['species'] = [filename.split('/')[0] for filename in train.filename]
train['record'] = [filename.split('/')[1] for filename in train.filename]
train['secondary_labels'] = [eval(sls) for sls in train['secondary_labels']]

In [ ]:
df = train.groupby('record').size()
df = df[df > 1]
df

In [ ]:
df = train.groupby('record').agg({'species' : ['first', 'last'],
                                  'secondary_labels': ['first', 'last'],
                                 })
df.columns = ['first_species', 'last_species', 'first_secondary', 'last_secondary']
df = df.reset_index()
df

In [ ]:
train = train.merge(df[['record', 'first_species', 'last_species',]],
                   how='left',
                   on='record')

In [ ]:
dups = {
    ('asbfly/XC724266.ogg', 'asbfly/XC724148.ogg'),
    ('barswa/XC575749.ogg', 'barswa/XC575747.ogg'),
    ('bcnher/XC669544.ogg', 'bcnher/XC669542.ogg'),
    ('bkskit1/XC350251.ogg', 'bkskit1/XC350249.ogg'),
    ('blhori1/XC417215.ogg', 'blhori1/XC417133.ogg'),
    ('blhori1/XC743616.ogg', 'blhori1/XC537503.ogg'),
    ('blrwar1/XC662286.ogg', 'blrwar1/XC662285.ogg'),
    ('brakit1/XC743675.ogg', 'brakit1/XC537471.ogg'),
    ('brcful1/XC197746.ogg', 'brcful1/XC157971.ogg'),
    ('brnshr/XC510751.ogg', 'brnshr/XC510750.ogg'),
    ('btbeat1/XC665307.ogg', 'btbeat1/XC513403.ogg'),
    ('btbeat1/XC743618.ogg', 'btbeat1/XC683300.ogg'),
    #('btbeat1/XC743619.ogg', 'btbeat1/XC683300.ogg'),
    ('btbeat1/XC743618.ogg', 'btbeat1/XC743619.ogg'),
    ('categr/XC787914.ogg', 'categr/XC438523.ogg'),
    ('cohcuc1/XC253418.ogg', 'cohcuc1/XC241127.ogg'),
    ('cohcuc1/XC423422.ogg', 'cohcuc1/XC423419.ogg'),
    ('comgre/XC202776.ogg', 'comgre/XC192404.ogg'),
    ('comgre/XC602468.ogg', 'comgre/XC175341.ogg'),
    ('comgre/XC64628.ogg', 'comgre/XC58586.ogg'),
    ('comior1/XC305930.ogg', 'comior1/XC303819.ogg'),
    ('comkin1/XC207123.ogg', 'comior1/XC207062.ogg'),
    ('comkin1/XC691421.ogg', 'comkin1/XC690633.ogg'),
    ('commyn/XC577887.ogg', 'commyn/XC577886.ogg'),
    ('commyn/XC652903.ogg', 'commyn/XC652901.ogg'),
    ('compea/XC665320.ogg', 'compea/XC644022.ogg'),
    ('comsan/XC385909.ogg', 'comsan/XC385908.ogg'),
    ('comsan/XC643721.ogg', 'comsan/XC642698.ogg'),
    ('comsan/XC667807.ogg', 'comsan/XC667806.ogg'),
    ('comtai1/XC126749.ogg', 'comtai1/XC122978.ogg'),
    ('comtai1/XC305210.ogg', 'comtai1/XC304811.ogg'),
    ('comtai1/XC542375.ogg', 'comtai1/XC540351.ogg'),
    ('comtai1/XC542379.ogg', 'comtai1/XC540352.ogg'),
    ('crfbar1/XC615780.ogg', 'crfbar1/XC615778.ogg'),
    ('dafbab1/XC188307.ogg', 'dafbab1/XC187059.ogg'),
    ('dafbab1/XC188308.ogg', 'dafbab1/XC187068.ogg'),
    ('dafbab1/XC188309.ogg', 'dafbab1/XC187069.ogg'),
    ('dafbab1/XC197745.ogg', 'dafbab1/XC157972.ogg'),
    ('eaywag1/XC527600.ogg', 'eaywag1/XC527598.ogg'),
    ('eucdov/XC355153.ogg', 'eucdov/XC355152.ogg'),
    ('eucdov/XC360303.ogg', 'eucdov/XC347428.ogg'),
    ('eucdov/XC365606.ogg', 'eucdov/XC124694.ogg'),
    ('eucdov/XC371039.ogg', 'eucdov/XC368596.ogg'),
    ('eucdov/XC747422.ogg', 'eucdov/XC747408.ogg'),
    ('eucdov/XC789608.ogg', 'eucdov/XC788267.ogg'),
    ('goflea1/XC163901.ogg', 'bladro1/XC163901.ogg'),
    ('goflea1/XC208794.ogg', 'bladro1/XC208794.ogg'),
    ('goflea1/XC208795.ogg', 'bladro1/XC208795.ogg'),
    ('goflea1/XC209203.ogg', 'bladro1/XC209203.ogg'),
    ('goflea1/XC209549.ogg', 'bladro1/XC209549.ogg'),
    ('goflea1/XC209564.ogg', 'bladro1/XC209564.ogg'),
    ('graher1/XC357552.ogg', 'graher1/XC357551.ogg'),
    ('graher1/XC590235.ogg', 'graher1/XC590144.ogg'),
    ('grbeat1/XC304004.ogg', 'grbeat1/XC303999.ogg'),
    ('grecou1/XC365426.ogg', 'grecou1/XC365425.ogg'),
    ('greegr/XC247286.ogg', 'categr/XC197438.ogg'),
    ('grewar3/XC743681.ogg', 'grewar3/XC537475.ogg'),
    ('grnwar1/XC197744.ogg', 'grnwar1/XC157973.ogg'),
    ('grtdro1/XC651708.ogg', 'grtdro1/XC613192.ogg'),
    ('grywag/XC459760.ogg', 'grywag/XC457124.ogg'),
    ('grywag/XC575903.ogg', 'grywag/XC575901.ogg'),
    ('grywag/XC650696.ogg', 'grywag/XC592019.ogg'),
    ('grywag/XC690448.ogg', 'grywag/XC655063.ogg'),
    ('grywag/XC745653.ogg', 'grywag/XC745650.ogg'),
    ('grywag/XC812496.ogg', 'grywag/XC812495.ogg'),
    ('heswoo1/XC357155.ogg', 'heswoo1/XC357149.ogg'),
    ('heswoo1/XC744698.ogg', 'heswoo1/XC665715.ogg'),
    ('hoopoe/XC631301.ogg', 'hoopoe/XC365530.ogg'),
    ('hoopoe/XC631304.ogg', 'hoopoe/XC252584.ogg'),
    ('houcro1/XC744704.ogg', 'houcro1/XC683047.ogg'),
    ('houspa/XC326675.ogg', 'houspa/XC326674.ogg'),
    ('inbrob1/XC744708.ogg', 'inbrob1/XC744706.ogg'),
    ('insowl1/XC305214.ogg', 'insowl1/XC301142.ogg'),
    ('junbab2/XC282587.ogg', 'junbab2/XC282586.ogg'),
    ('labcro1/XC267645.ogg', 'labcro1/XC265731.ogg'),
    ('labcro1/XC345836.ogg', 'labcro1/XC312582.ogg'),
    ('labcro1/XC37773.ogg', 'labcro1/XC19736.ogg'),
    ('labcro1/XC447036.ogg', 'houcro1/XC447036.ogg'),
    ('labcro1/XC823514.ogg', 'gybpri1/XC823527.ogg'),
    ('laudov1/XC185511.ogg', 'grewar3/XC185505.ogg'),
    ('laudov1/XC405375.ogg', 'laudov1/XC405374.ogg'),
    ('laudov1/XC514027.ogg', 'eucdov/XC514027.ogg'),
    ('lblwar1/XC197743.ogg', 'lblwar1/XC157974.ogg'),
    ('lewduc1/XC261506.ogg', 'lewduc1/XC254813.ogg'),
    ('litegr/XC403621.ogg', 'bcnher/XC403621.ogg'),
    ('litegr/XC535540.ogg', 'litegr/XC448898.ogg'),
    ('litegr/XC535552.ogg', 'litegr/XC447850.ogg'),
    ('litgre1/XC630775.ogg', 'litgre1/XC630560.ogg'),
    ('litgre1/XC776082.ogg', 'litgre1/XC663244.ogg'),
    ('litspi1/XC674522.ogg', 'comtai1/XC674522.ogg'),
    ('litspi1/XC722435.ogg', 'litspi1/XC721636.ogg'),
    ('litspi1/XC722436.ogg', 'litspi1/XC721637.ogg'),
    ('litswi1/XC443070.ogg', 'litswi1/XC440301.ogg'),
    ('lobsun2/XC197742.ogg', 'lobsun2/XC157975.ogg'),
    ('maghor2/XC197740.ogg', 'maghor2/XC157978.ogg'),
    ('maghor2/XC786588.ogg', 'maghor2/XC786587.ogg'),
    ('malpar1/XC197770.ogg', 'malpar1/XC157976.ogg'),
    ('marsan/XC383290.ogg', 'marsan/XC383288.ogg'),
    ('marsan/XC733175.ogg', 'marsan/XC716673.ogg'),
    ('mawthr1/XC455222.ogg', 'mawthr1/XC455211.ogg'),
    ('orihob2/XC557991.ogg', 'orihob2/XC557293.ogg'),
    ('piebus1/XC165050.ogg', 'piebus1/XC122395.ogg'),
    ('piebus1/XC814459.ogg', 'piebus1/XC792272.ogg'),
    ('placuc3/XC490344.ogg', 'placuc3/XC486683.ogg'),
    ('placuc3/XC572952.ogg', 'placuc3/XC572950.ogg'),
    ('plaflo1/XC615781.ogg', 'plaflo1/XC614946.ogg'),
    ('purher1/XC467373.ogg', 'graher1/XC467373.ogg'),
    ('purher1/XC827209.ogg', 'purher1/XC827207.ogg'),
    ('pursun3/XC268375.ogg', 'comtai1/XC241382.ogg'),
    ('pursun4/XC514853.ogg', 'pursun4/XC514852.ogg'),
    ('putbab1/XC574864.ogg', 'brcful1/XC574864.ogg'),
    ('rewbul/XC306398.ogg', 'bkcbul1/XC306398.ogg'),
    ('rewbul/XC713308.ogg', 'asbfly/XC713467.ogg'),
    ('rewlap1/XC733007.ogg', 'rewlap1/XC732874.ogg'),
    ('rorpar/XC199488.ogg', 'rorpar/XC199339.ogg'),
    ('rorpar/XC402325.ogg', 'comior1/XC402326.ogg'),
    ('rorpar/XC516404.ogg', 'rorpar/XC516402.ogg'),
    ('sbeowl1/XC522123.ogg', 'brfowl1/XC522123.ogg'),
    ('sohmyn1/XC744700.ogg', 'sohmyn1/XC743682.ogg'),
    ('spepic1/XC804432.ogg', 'spepic1/XC804431.ogg'),
    ('spodov/XC163930.ogg', 'bladro1/XC163901.ogg'),
    ('spodov/XC163930.ogg', 'goflea1/XC163901.ogg'),
    ('spoowl1/XC591485.ogg', 'spoowl1/XC591177.ogg'),
    ('stbkin1/XC266782.ogg', 'stbkin1/XC266682.ogg'),
    ('stbkin1/XC360661.ogg', 'stbkin1/XC199815.ogg'),
    ('stbkin1/XC406140.ogg', 'stbkin1/XC406138.ogg'),
    ('vefnut1/XC197738.ogg', 'vefnut1/XC157979.ogg'),
    ('vefnut1/XC293526.ogg', 'vefnut1/XC289785.ogg'),
    ('wemhar1/XC581045.ogg', 'comsan/XC581045.ogg'),
    ('wemhar1/XC590355.ogg', 'wemhar1/XC590354.ogg'),
    ('whbbul2/XC335671.ogg', 'whbbul2/XC335670.ogg'),
    ('whbsho3/XC856465.ogg', 'whbsho3/XC856463.ogg'),
    #('whbsho3/XC856468.ogg', 'whbsho3/XC856463.ogg'),
    ('whbsho3/XC856465.ogg', 'whbsho3/XC856468.ogg'),
    ('whbwat1/XC840073.ogg', 'whbwat1/XC840071.ogg'),
    ('whbwoo2/XC239509.ogg', 'rufwoo2/XC239509.ogg'),
    ('whcbar1/XC659329.ogg', 'insowl1/XC659329.ogg'),
    ('whiter2/XC265271.ogg', 'whiter2/XC265267.ogg'),
    ('whtkin2/XC197737.ogg', 'whtkin2/XC157981.ogg'),
    ('whtkin2/XC430267.ogg', 'whtkin2/XC430256.ogg'),
    ('whtkin2/XC503389.ogg', 'comior1/XC503389.ogg'),
    ('whtkin2/XC540094.ogg', 'whtkin2/XC540087.ogg'),
    ('woosan/XC184466.ogg', 'marsan/XC184466.ogg'),
    ('woosan/XC545316.ogg', 'woosan/XC476064.ogg'),
    ('woosan/XC587076.ogg', 'woosan/XC578599.ogg'),
    ('woosan/XC742927.ogg', 'woosan/XC740798.ogg'),
    ('woosan/XC825766.ogg', 'grnsan/XC825765.ogg'),
    ('zitcis1/XC303866.ogg', 'zitcis1/XC302781.ogg'),
}

In [ ]:
def to_remove(r0, r1):
    name0 = r0.split('/')[0]
    name1 = r1.split('/')[0]
    return name0 == name1

In [ ]:
dups = [(r0,r1) for (r0,r1) in dups if to_remove(r0, r1)]
len(dups)

In [ ]:
to_remove = set(r1 for r0,r1 in dups)
len(to_remove)

In [ ]:
train = train[~train.filename.isin(to_remove)].reset_index(drop=True)

In [ ]:
train = train.groupby('record').first().reset_index()

In [ ]:
train['record_name'] = [record.split('.')[0] for record in train.record]

In [ ]:
train

In [ ]:
all_train = pd.read_csv('../all_train.csv')
all_train

In [ ]:
all_train.groupby(['primary_label', 'source']).size()

In [ ]:
all_train.source.value_counts()

In [ ]:
all_train[all_train.secondary_labels != '[]'].source.value_counts()

In [ ]:
all_train['species'] = all_train['primary_label']
all_train['first_species'] = all_train['primary_label']
all_train['last_species'] = all_train['primary_label']
all_train['filename'] = [ species + '/' + record + '.npy' for species, record in zip(all_train['primary_label'], all_train['record_name'])]
all_train['secondary_labels'] = [eval(sls) for sls in all_train['secondary_labels']]
all_train

In [ ]:
def load_audio(filename, first, istrain, cfg):
    filepath = filename.split('/')[0]
    fname = filename.split('/')[1].split('.')[0]
    filepath = cfg.input_path / 'birdclef_data' / filepath 
    if istrain:
        max_duration =  int((cfg.duration + cfg.max_shift) * cfg.sr)
    else:
        max_duration = cfg.duration * cfg.sr
    if first:
        filepath = filepath / f"first10_{fname}.npy"
        audio = np.load(filepath)
        audio = audio[:max_duration]
    else:
        filepath = filepath / f"last10_{fname}.npy"
        audio = np.load(filepath)
        audio = audio[-max_duration:]
    return audio

In [ ]:
load_audio('asbfly/XC134896.ogg', True, True, cfg).shape

In [ ]:
load_audio('crseag1/XC88411.mp3', True, True, cfg).shape

In [ ]:
load_audio('zitcis1/XC124995.npy', True, True, cfg).shape

In [ ]:
len(set(train.record_name) & set(all_train.record_name))

In [ ]:
train = pd.concat([train, all_train])

In [ ]:
train

In [ ]:
new_train = []

kf = KFold(n_splits=cfg.num_folds, shuffle=True, random_state=0)
for species, df in train.groupby('primary_label'):
    df = df.reset_index(drop=True)
    df['fold'] = -1
    for fold, (train_index, valid_index) in enumerate(kf.split(df, df.primary_label)):
        df.loc[valid_index, "fold"] = int(fold)
    new_train.append(df)
new_train = pd.concat(new_train).reset_index(drop=True)    
new_train.fold.value_counts()

In [ ]:
new_train.groupby(['species', 'fold']).size().unstack()

In [ ]:
train = new_train

In [ ]:
train

In [ ]:
def metric(preds, targets, cfg):
    score = {}
    for j,label in enumerate(cfg.labels):
        score[label] = roc_auc_score(targets[:, j], preds[:, j])
    score_avg = np.mean([v for k,v in score.items()])
    return score_avg, score

def metric_db(train, oofs, cfg):
    score = {}
    for j,label in enumerate(cfg.labels):
        score[label] = roc_auc_score(train.primary_label == label, oofs[label])
    score_avg = np.mean([v for k,v in score.items()])
    return score_avg, score
    
def my_softmax(preds):
    preds = preds - preds.max(1, keepdims=True)
    preds = np.exp(preds.clip(-20, 0))
    preds = preds / preds.sum(1, keepdims=True)
    return preds

def bce_with_mask(preds, targets, mask):
    loss = nn.BCEWithLogitsLoss(reduction='none')(preds, targets)
    loss = loss * mask
    loss = loss.mean()
    return loss
    

In [ ]:
cfg.device = torch.device('cuda')
if cfg.bce:
    #cfg.loss = nn.BCEWithLogitsLoss()
    cfg.loss = bce_with_mask
else:
    cfg.loss = nn.CrossEntropyLoss()

cfg.labels = np.array(sorted(train.species.unique()))
cfg.num_labels = len(cfg.labels)
cfg.targets = {v : i for i,v in enumerate(cfg.labels)}

cfg.logger = get_logger(cfg)
seed_torch(cfg.seed)

In [ ]:
class BirdDataset():
    def __init__(self, train, istrain, cfg):
        self.cfg = cfg
        self.istrain = istrain
        self.filename = train.filename.values
        #self.primary_label = train.primary_label.values
        self.secondary_labels = train.secondary_labels.values
        self.first_species = train.first_species.values
        self.last_species = train.last_species.values
        
    def __len__(self):
        return len(self.filename)
    
    def get_audio(self, idx):
        filename = self.filename[idx]
        duration = self.cfg.sr * self.cfg.duration
        if self.istrain:
            first = np.random.rand() < 0.5
            audio = load_audio(filename, first, True, self.cfg)
            if len(audio) < duration:
                pad_length = np.random.randint(0, duration - len(audio) + 1) 
                audio = np.pad(audio, 
                               ((pad_length, duration - len(audio) - pad_length),), 
                               mode='constant')
            else:
                start = np.random.randint(0, len(audio) - duration + 1)
                audio = audio[start : start + duration]
        else:
            audio = load_audio(filename, True, False, self.cfg)
            audio = audio[:duration]
            if len(audio) < duration:
                pad_length = (duration - len(audio)) // 2
                audio = np.pad(audio, 
                               ((pad_length, duration - len(audio) - pad_length),), 
                               mode='constant')
        return audio
      
    def __getitem__(self, idx):
        audio = self.get_audio(idx)
        targets = np.zeros(len(cfg.labels), dtype=np.float32)        
        targets[self.cfg.targets[self.first_species[idx]]] = 1.0
        targets[self.cfg.targets[self.last_species[idx]]] = 1.0
        secondary_mask = np.ones(len(cfg.labels), dtype=np.float32)
        secondary_labels = self.secondary_labels[idx]
        if len(secondary_labels) > 0:
            for label in secondary_labels:
                if label in cfg.targets:
                    secondary_mask[cfg.targets[label]] = 0
        if self.istrain and self.cfg.other_samples:
            num_samples = np.random.randint(0, self.cfg.other_samples + 1)
            for _ in range(num_samples):
                other_idx = np.random.randint(len(self.filename))
                other_audio = self.get_audio(other_idx)
                weight = 0.2 + 0.8 *  np.random.rand()
                audio += weight * other_audio
                targets[self.cfg.targets[self.first_species[other_idx]]] = 1.0
                targets[self.cfg.targets[self.last_species[other_idx]]] = 1.0
                secondary_labels = self.secondary_labels[other_idx]
                if len(secondary_labels) > 0:
                    for label in secondary_labels:
                        if label in cfg.targets:
                            secondary_mask[cfg.targets[label]] = 0
        secondary_mask = np.maximum(secondary_mask, targets)        
        out = {
            'audio' : torch.from_numpy(audio),
            'targets' : torch.from_numpy(targets),
            'secondary_mask' : secondary_mask,
        }
        return out

def batch_to_device(batch, device):
    return {k:batch[k].to(device, non_blocking=True) for k in batch.keys() if k not in []}

In [ ]:
dataset = BirdDataset(train, True, cfg)
elt = dataset[0]
for k,v in elt.items():
    print(k, v.shape)

In [ ]:
def get_data_loader(dataset, istrain, cfg):
    if istrain:
        batch_size = cfg.train_batch_size
    else:
        batch_size = cfg.valid_batch_size
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=cfg.workers,
        shuffle=istrain,
        pin_memory=False,
        #collate_fn=collate_pad,
        drop_last = istrain,
    )
    return data_loader 

In [ ]:
def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, p_trainable=True):
        super(GeM,self).__init__()
        if p_trainable:
            self.p = Parameter(torch.ones(1)*p)
        else:
            self.p = p
        self.eps = eps

    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)       
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

class BirdModel(nn.Module):
    def __init__(self, cfg, pretrained: bool = True):
        super(BirdModel, self).__init__()
        self.cfg = cfg
        self.mel = T.MelSpectrogram(
            sample_rate=cfg.sr, n_fft=cfg.n_fft, win_length=cfg.win_length, 
            hop_length= cfg.hop_length, f_min=cfg.fmin, f_max=cfg.fmax, 
            n_mels=cfg.n_mels, mel_scale='htk', power=2.0)
        self.A2DB = T.AmplitudeToDB(stype="power")
        self.backbone = timm.create_model(
            cfg.backbone,
            pretrained=pretrained,
            drop_rate = 0.1,
            #drop_path_rate = 0.2,
            num_classes=cfg.num_labels, 
            #global_pool=''
        )
        #if cfg.gem_pooling == "gem":
        #    self.backbone.head.global_pool = GeM(p_trainable=args.p_trainable)
         
    def forward(self, input_dict):
        x = input_dict['audio']
        with autocast(enabled=False), torch.no_grad():
            x = x / torch.std(x, 1, keepdim=True)
            x = x.float()
            x = self.mel(x)
            x = self.A2DB(x)
            x = (x - 40) / 80
        with torch.no_grad():
            x = x.unsqueeze(1)
            pos = torch.linspace(0., 1., x.size(2)).to(x.device)
            pos = pos.unsqueeze(0).unsqueeze(0).unsqueeze(-1)
            pos = pos.expand(x.size(0), 1, x.size(2), x.size(3))
            x = x.expand(-1, 2, -1, -1)
            x = torch.cat([x, pos], 1)

        x = self.backbone(x)
        return x

In [ ]:
data_loader = get_data_loader(dataset, False, cfg)
model = BirdModel(cfg)
for batch in data_loader:
    break
out = model(batch)
out.shape

In [ ]:
def train_epoch(loader, model, optimizer, scheduler, scaler, device, cfg):
    model.train()
    model.zero_grad()
    if cfg.verbose:
        bar = tqdm(range(len(loader)))
    else:
        bar = range(len(loader))
    load_iter = iter(loader)
    loss_l = []
    grad_norm_l = []
    
    accumulate = cfg.accumulate
    
    for i, batch in zip(bar, load_iter):
        input_dict = batch_to_device(batch, device)
        with autocast(enabled=True):
            targets = input_dict['targets']
            if cfg.loudness_range:
                loudness = - np.log(cfg.loudness_range)
                bs = targets.shape[0]
                #weight = 0.1 ** (cfg.loundness_range *  np.random.rand() / 10)
                weight = torch.rand(bs, 1).to(targets.device)
                weight = torch.exp(weight * loudness)
                audio = input_dict['audio']
                audio = audio * weight
                input_dict['audio'] = audio
            preds = model(input_dict)
            if cfg.bce:
                secondary_mask = input_dict['secondary_mask']
                loss = cfg.loss(preds, targets, secondary_mask).mean()
            else:
                loss = cfg.loss(preds, targets).mean()
        loss_l.append(loss.detach().cpu().item())
        scaler.scale(loss / cfg.accumulate).backward() 
        accumulate -= 1
        if accumulate == 0:
            if cfg.grad_value:
                scaler.unscale_(optimizer)
                nn.utils.clip_grad_value_(model.parameters(), cfg.grad_value)
            if cfg.grad_norm:
                scaler.unscale_(optimizer)
                total_norm = nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_norm).item()
                if np.isnan(total_norm):
                    total_norm = cfg.grad_norm
                else:
                    total_norm = np.clip(total_norm, 0, cfg.grad_norm)
                grad_norm_l.append(total_norm)
            scaler.step(optimizer)     
            scaler.update()
            optimizer.zero_grad()
            accumulate = cfg.accumulate
            scheduler.step()  
        del preds, targets, loss, input_dict
        if cfg.verbose:
            if cfg.grad_norm:
                bar.set_description('loss: %.4f grad norm %.1f' % (np.mean(loss_l), np.mean(grad_norm_l),))
            else:
                bar.set_description('loss: %.4f ' % np.mean(loss_l))
    optimizer.zero_grad()
    del loss_l, bar
    gc.collect()

In [ ]:
def valid_epoch(loader, model, device, cfg):
    model.eval()
    model.zero_grad()
    if cfg.verbose:
        bar = tqdm(range(len(loader)))
    else:
        bar = range(len(loader))
    load_iter = iter(loader)
    preds_l = []
    targets_l = []
    with torch.no_grad():
        for i, batch in zip(bar, load_iter):      
            input_dict = batch_to_device(batch, device)
            with autocast(enabled=False):
                preds = model(input_dict)                
            preds_l.append(preds.detach().cpu())
            del preds, input_dict
            targets = batch['targets']
            targets_l.append(targets)
        preds = torch.cat(preds_l)
        targets = torch.cat(targets_l)
        return preds.numpy(), targets.numpy()

In [ ]:
def get_optimizer(model, cfg):
    no_decay = ["bias", "norm"]
    if cfg.no_decay:
        optimizer_parameters = [
            {'params': [p for n, p in model.named_parameters() 
                        if not any(nd in n for nd in no_decay)],
             'weight_decay': cfg.decay},
            {'params': [p for n, p in model.named_parameters() 
                        if any(nd in n for nd in no_decay)],
             'weight_decay': 0.0},
        ] 
    else:
        optimizer_parameters = model.parameters()
    optimizer = torch.optim.AdamW(optimizer_parameters, lr=cfg.lr)
    return optimizer

def get_scheduler(optimizer, train_data_loader, cfg):
    scheduler = lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=cfg.max_lr,
        epochs=cfg.num_epochs,
        steps_per_epoch=len(train_data_loader),
        pct_start=cfg.pct_start,
        anneal_strategy="cos",
        final_div_factor=cfg.final_div_factor,
    )
    return scheduler

In [ ]:
def save_checkpoint(model, fold, seed, cfg):
    checkpoint = {
        'model' : model.state_dict(),
        'fold' : fold,
        'seed' : seed,
        }
    checkpoint_path = cfg.checkpoint_path
    save_path = checkpoint_path / ('%s_%d_%d.pt' % (cfg.fname, fold, seed))
    if cfg.local_rank == 0:
        cfg.logger.info('saving %s ...' % save_path)
    torch.save(checkpoint, save_path)
    if cfg.local_rank == 0:
        cfg.logger.info('done')

def load_checkpoint(fold, seed, cfg):
    if cfg.pretrained_path:
        checkpoint_path = cfg.pretrained_path
    else:
        checkpoint_path = cfg.checkpoint_path
    save_path = checkpoint_path / ('%s_%d_%d.pt' % (cfg.fname, fold, seed, ))
    cfg.logger.info('loading %s ...' % save_path)
    checkpoint = torch.load(save_path, map_location='cpu')
    model = BirdModel(cfg, pretrained=False).to(cfg.device)
    model.load_state_dict(checkpoint['model'], strict=True)
    model.eval()
    cfg.logger.info('done')
    return model


In [ ]:
def resample(train_fold, cfg):
    new_train = []
    for species, df in train.groupby('species'):
        new_train.append(df)
        if len(df) < cfg.resample_train:
            df = df.sample(n=(cfg.resample_train - len(df)), replace=True, random_state=cfg.seed)
            new_train.append(df)
    new_train = pd.concat(new_train).reset_index(drop=True)  
    return new_train  

In [ ]:
scores = []
oofs = pd.DataFrame(columns=cfg.labels, index=train.index, data = 0.0)
oofs['filename'] = train.filename

In [ ]:
for fold in range(cfg.num_folds):
    if cfg.fold >= 0 and cfg.fold != fold:
        continue
    seed = cfg.seed + fold
    seed_torch(seed)
    if cfg.pl is not None:
        pl_train = get_pl_train(cfg)
    else:
        pl_train = None
        train_fold = train[train.fold != fold]
    valid_dataset = BirdDataset(train[train.fold == fold], False, cfg)
    valid_dataloader = get_data_loader(valid_dataset, istrain=False, cfg=cfg)
    device = cfg.device
    
    if cfg.pretrained_path:
        model = load_checkpoint(fold, seed, cfg)
    else:
        model = BirdModel(cfg, pretrained=True).to(device)
    optimizer = get_optimizer(model, cfg)
    scheduler = None
    scaler = GradScaler()
    result = None
    for epoch in range(cfg.num_epochs):
        if cfg.pl is not None:
            train_fold = sample_pl(train, fold, pl_train)
        if cfg.resample_train:
            train_fold = resample(train_fold, cfg)
        train_dataset = BirdDataset(train_fold, True, cfg)
        train_dataloader = get_data_loader(train_dataset, istrain=True, cfg=cfg)

        if scheduler is None:
            scheduler = get_scheduler(optimizer, train_dataloader, cfg)
        train_epoch(train_dataloader, model, optimizer, scheduler, scaler, device, cfg)
        if valid_dataset is not None:
            preds, targets = valid_epoch(valid_dataloader, model, device, cfg)
            if cfg.bce:
                preds = expit(preds) # model uses logits
            else:
                preds = my_softmax(preds) # model uses logits
            result, _ = metric(preds, targets, cfg)
            msg = f"seed {cfg.seed} fold {fold} epoch {epoch} metric {result:.4f}"
            cfg.logger.info(msg)
        else:
            msg = f"seed {cfg.seed} fold {fold} epoch {epoch}"
            cfg.logger.info(msg)
    if cfg.local_rank == 0:
        save_checkpoint(model, fold, seed, cfg)
    del model, optimizer, scheduler, scaler, train_dataloader, 
    if valid_dataset is not None:
        del valid_dataloader
    gc.collect()
    torch.cuda.empty_cache()
    scores.append(result)
    
    for j,c in enumerate(cfg.labels):
        oofs.loc[train.fold == fold, c] = preds[:, j]
oofs.to_csv(cfg.checkpoint_path / 'oofs.csv', index=False)

np.mean(scores), metric_db(train, oofs, cfg)

In [ ]:
res = metric_db(train, oofs, cfg)
res

In [ ]:
dfe = train.groupby('primary_label').size()
dfe.name = 'size'
dfe = dfe.reset_index().sort_values('primary_label').reset_index(drop=True)
dfe

In [ ]:
df = pd.DataFrame({'label' : cfg.labels, 
                   'roc_auc' : [res[1][label] for label in cfg.labels],
                   'size' : dfe['size'].values,
                  })
df

In [ ]:
plt.scatter(np.log(df['size']), df['roc_auc'], marker='+')

In [ ]:
df.sort_values('roc_auc')

In [ ]:
df = train.groupby('primary_label').size()
df = df / df.sum()
df.name = 'avg'
df = df.to_frame()
df['oof'] = oofs[cfg.labels].mean(0)
df

In [ ]:
plt.scatter(df.avg, df.oof/df.avg, marker='+')

In [ ]:
class SoundScapeDataset():
    def __init__(self, soundscape, istrain, cfg):
        self.cfg = cfg
        self.istrain = istrain
        self.soundscape = soundscape
        self.duration = int(np.round(soundscape.shape[0] / cfg.sr))
        self.starts = np.arange(0, self.duration, cfg.duration)
        self.ends = self.starts + cfg.duration
        self.starts = cfg.sr * self.starts
        self.ends = cfg.sr * self.ends
        
    def __len__(self):
        return len(self.starts)
    
    def __getitem__(self, idx):
        audio = self.soundscape[self.starts[idx] : self.ends[idx]]
        duration = self.cfg.sr * self.cfg.duration
        if len(audio) < duration:
            pad_length = (duration - len(audio)) // 2
            audio = np.pad(audio, 
                           ((pad_length, duration - len(audio) - pad_length),), 
                           mode='constant')
        out = {
            'audio' : torch.from_numpy(audio),
        }
        return out

def predict(loader, models, cfg):
    for model in models:
        model.eval()
        model.zero_grad()
    if cfg.verbose:
        bar = tqdm(range(len(loader)))
    else:
        bar = range(len(loader))
    load_iter = iter(loader)
    preds_l = [[] for model in models]
    targets_l = []
    with torch.no_grad():
        for i, batch in zip(bar, load_iter):      
            input_dict = batch_to_device(batch, cfg.device)
            with autocast(enabled=False):
                for preds, model in zip(preds_l, models):
                    preds.append(model(input_dict).detach().cpu()) 
            del input_dict
        preds_l = [torch.cat(preds).squeeze().numpy() for preds in preds_l]
        return preds_l

def load_soundscape(pathname, cfg):
    #audio = librosa.load(pathname, sr=32000)[0].astype(np.float32)
    filename = pathname.split('/')[-1].split('.')[0]
    audio = np.load(cfg.input_path / 'birdclef_data' / 'unlabeled_soundscapes' / (filename + '.npy'))

    return audio

def predict_soundscape(filepath, models, cfg):
    waveform = load_soundscape(filepath, cfg)
    #return len(waveform) / cfg.sr
    #return waveform
    #print('duration',  len(waveform) / cfg.sr)
    dataset = SoundScapeDataset(waveform, False, cfg)
    dataloader = get_data_loader(dataset, False, cfg)
    preds_l = predict(dataloader, models, cfg)
    if cfg.bce:
        preds_l = [expit(preds) for preds in preds_l] # model uses logits
    else:
        preds_l = [my_softmax(preds) for preds in preds_l] # model uses logits
    return preds_l

In [ ]:
soundscapes = sorted(glob(str(cfg.soundscape_path / '*.ogg')))
soundscapes[:5]



In [ ]:
str(cfg.checkpoint_path)

In [ ]:
! cp ../checkpoints/bird_003/exp_79?/*.pt ../checkpoints/bird_003/exp_790

In [ ]:
!ls  ../checkpoints/bird_003/exp_790

In [ ]:
cfg.verbose = False
cfg.workers = 0
cfg.valid_batch_size = 128

models = [load_checkpoint(fold, cfg.seed + fold, cfg).to(cfg.device)
          for fold in range(cfg.num_folds)
         ]
preds = {}
for soundscape in tqdm(soundscapes):
    preds[soundscape]  = predict_soundscape(soundscape, models, cfg)
with open(cfg.checkpoint_path / f"pl_all.pkl", "wb") as file:
    pkl.dump(preds, file)